In [3]:
import pandas as pd
import numpy as np

In [4]:
# 打者の聞き手属性:右,左
BATTER_HANDEDNESS = 'batter_handedness'
BATTER_HANDEDNESS_ATTERRS = '右'

# 投手の聞き手属性:右,左
PITCHER_HANDEDNESS = 'pitcher_handedness'
PITCHER_HANDEDNESS_ATTERRS = '右'

# 球種:ストレート,スライダー,フォークカットボール,シュート,チェンジアップ,カーブ,シンカー,特殊球
PITCH_TYPE_NAME = 'pitch_type_name'
PITCH_TYPE_NAME_ATTRS = 'ストレート'

# コース:
#|21|19|17|15|10|
#|22|7 |4 |1 |11|
#|23|8 |5 |2 |12|
#|24|9 |6 |3 |13|
#|25|20|18|16|14|
PITCH_ZONE = 'pitch_zone'
SEARCH_COLUMN = '5'

# 球速:58.0〜163.0
PITCH_SPEED= 'pitch_speed'

#ストライク
IS_STRIKE = 'is_strike'

# 空振りフラグ:1
IS_CALLED_STRIKE = 'is_called_strike'

# 見逃しフラグ:1
IS_SWINGING_STRIKE = 'is_swinging_strike'

# スイングフラグ:1
IS_FOUL = 'is_foul'

# 牽制フラグ:1
PICKOFF_ATTEMPT_TO = 'pickoff_attempt_to'

In [9]:
# ===== データの読み込み =====
# train.csv: モデルを学習するためのデータ
df = pd.read_csv('/Users/takurateruyoshi/Downloads/work_dataset/play_info.csv', encoding='cp932')

# フィルタリングを実行
# 1. 比較対象の列を文字列に変換 (.astype(str))
# 2. 比較 (== SEARCH_ATTRTS)
# 3. フィルタリング (train_df[...])
print(f"データ件数: {len(df)}")
df = df[df[PICKOFF_ATTEMPT_TO].astype(str).isin(['0'])]  # 牽制球のみ
print(f"データ件数: {len(df)}")

/var/folders/6t/2qvj7grs6tx1rjydtydst6br0000gn/T/ipykernel_53652/2145991319.py:3: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/takurateruyoshi/Downloads/work_dataset/play_info.csv', encoding='cp932')


データ件数: 254361
データ件数: 247574


In [6]:
# 文字列 "1"/"0" や True/False を 1/0 に正規化
s = df[IS_STRIKE]

# 文字→数値に変換（失敗は NaN）
s_num = pd.to_numeric(s, errors="coerce")

# True/False を 1/0 に
s_num = s_num.fillna(s.where(s.isin([True, False])).astype("float"))

# 以降は同様
n1 = (s_num == 1).sum()
n0 = (s_num == 0).sum()
valid = n1 + n0
ratio_1_valid = n1 / valid if valid > 0 else float("nan")

print(f"1の数: {n1}, 0の数: {n0}, 1の割合(有効): {ratio_1_valid:.3f}")

1の数: 161593, 0の数: 85981, 1の割合(有効): 0.653


In [7]:
#ストライク率の計算
for 投手 in df[BATTER_HANDEDNESS].unique():
    df_投手 = df[df[BATTER_HANDEDNESS].astype(str).isin([投手])]
    #print(f"投手: {投手}")
    for 打者 in df_投手[PITCHER_HANDEDNESS].unique():
        df_投手_打者 = df_投手[df_投手[PITCHER_HANDEDNESS].astype(str).isin([打者])]
        #print(f" 打者: {打者}")
        for 球種 in df_投手_打者[df_投手_打者[PITCH_TYPE_NAME].notna()][PITCH_TYPE_NAME].unique():
            df_投手_打者_球種 = df_投手_打者[df_投手_打者[PITCH_TYPE_NAME].astype(str).isin([球種])]
            #print(f"  球種: {球種}")
            for コース in df_投手_打者_球種[df_投手_打者_球種[PITCH_ZONE].notna()][PITCH_ZONE].unique():
                df_投手_打者_球種_コース = df_投手_打者_球種[df_投手_打者_球種[PITCH_ZONE].astype(str).isin([str(コース)])]
                #print(f"   コース: {コース}")
                投球合計数 = len(df_投手_打者_球種_コース)
                print(f"投手: {投手}, 打者: {打者}, 球種: {球種}, コース: {コース}")
                for 球速 in range(60, 161, 10):
                    df_投手_打者_球種_コース_球速 = df_投手_打者_球種_コース[
                        (df_投手_打者_球種_コース[PITCH_SPEED] >= 球速) &
                        (df_投手_打者_球種_コース[PITCH_SPEED] < 球速 + 10)
                    ]
                    投球数 = len(df_投手_打者_球種_コース_球速)
                    if 投球数 == 0:
                        continue
                    ストライク数 = df_投手_打者_球種_コース_球速[IS_STRIKE].sum()
                    ストライク率 = ストライク数 / 投球数
                    print(f"    球速: {球速}〜{球速 + 9} mph, 投球数: {投球数}, ストライク数: {ストライク数}, ストライク率: {ストライク率:.3f}")
                    

投手: 左, 打者: 右, 球種: ストレート, コース: 2.0
    球速: 120〜129 mph, 投球数: 7, ストライク数: 7, ストライク率: 1.000
    球速: 130〜139 mph, 投球数: 50, ストライク数: 50, ストライク率: 1.000
    球速: 140〜149 mph, 投球数: 1543, ストライク数: 1543, ストライク率: 1.000
    球速: 150〜159 mph, 投球数: 972, ストライク数: 972, ストライク率: 1.000
    球速: 160〜169 mph, 投球数: 3, ストライク数: 3, ストライク率: 1.000
投手: 左, 打者: 右, 球種: ストレート, コース: 8.0
    球速: 110〜119 mph, 投球数: 1, ストライク数: 1, ストライク率: 1.000
    球速: 120〜129 mph, 投球数: 7, ストライク数: 7, ストライク率: 1.000
    球速: 130〜139 mph, 投球数: 32, ストライク数: 32, ストライク率: 1.000
    球速: 140〜149 mph, 投球数: 1010, ストライク数: 1010, ストライク率: 1.000
    球速: 150〜159 mph, 投球数: 692, ストライク数: 692, ストライク率: 1.000
    球速: 160〜169 mph, 投球数: 3, ストライク数: 3, ストライク率: 1.000
投手: 左, 打者: 右, 球種: ストレート, コース: 3.0
    球速: 120〜129 mph, 投球数: 10, ストライク数: 10, ストライク率: 1.000
    球速: 130〜139 mph, 投球数: 40, ストライク数: 40, ストライク率: 1.000
    球速: 140〜149 mph, 投球数: 1195, ストライク数: 1195, ストライク率: 1.000
    球速: 150〜159 mph, 投球数: 716, ストライク数: 716, ストライク率: 1.000
    球速: 160〜169 mph, 投球数: 1, ストライク数: 1, ストライク率: 1.

In [29]:
import json
import numpy as np
import pandas as pd

# --- 定数の定義 ---
BATTER_HANDEDNESS = 'batter_handedness'
PITCHER_HANDEDNESS = 'pitcher_handedness'
PITCH_TYPE_NAME = 'pitch_type_name'
PITCH_ZONE = 'pitch_zone'
PITCH_SPEED = 'pitch_speed'
IS_STRIKE = 'is_strike'
IS_CALLED_STRIKE = 'is_called_strike'
IS_SWINGING_STRIKE = 'is_swinging_strike'
IS_FOUL = 'is_foul'

IS_H = 'is_h'                # 安打フラグ 
IS_PITCH = 'is_pitch'        # 投球フラグ 

# ウェブアプリ分析のために内部で使うカスタムフラグ
IS_SWINGING_CUSTOM = 'is_swinging_custom'
IS_LOOKING_CUSTOM = 'is_looking_custom'   

# CSVゾーン番号からWebアプリの5x5行列インデックス (0-24) へのマッピング
CSV_TO_WEB_INDEX_MAP = {
    # r=0 (高)
    '21.0': 0, '19.0': 1, '17.0': 2, '15.0': 3, '10.0': 4,
    # r=1 (中高)
    '22.0': 5, '7.0': 6, '4.0': 7, '1.0': 8, '11.0': 9,
    # r=2 (中心)
    '23.0': 10, '8.0': 11, '5.0': 12, '2.0': 13, '12.0': 14,
    # r=3 (中低)
    '24.0': 15, '9.0': 16, '6.0': 17, '3.0': 18, '13.0': 19,
    # r=4 (低)
    '25.0': 20, '20.0': 21, '18.0': 22, '16.0': 23, '14.0': 24
}

# --------------------------------------------------------
# 🎯 データフレームのロードと前処理
# --------------------------------------------------------
try:
    # 実際のデータロード処理 (パスは環境に合わせてください)
    df = pd.read_csv('/Users/takurateruyoshi/Downloads/work_dataset/play_info.csv', encoding='cp932')
    
    # 🎯 修正点1: PITCH_ZONE列の型変換と空白文字削除を確実に行う
    df[PITCH_ZONE] = df[PITCH_ZONE].astype(str).str.strip()
    print(f"PITCH_ZONE unique values after stripping: {df[PITCH_ZONE].unique()}")
    # カスタムフラグの作成
    df[IS_SWINGING_CUSTOM] = ((df[IS_SWINGING_STRIKE] == 1) | (df[IS_FOUL] == 1)).astype(int)
    df[IS_LOOKING_CUSTOM] = ((df[IS_CALLED_STRIKE] == 1) & (df[IS_SWINGING_CUSTOM] == 0)).astype(int)

except Exception as e:
    print(f"🚨 データロードまたは前処理中にエラーが発生しました: {e}")
    print("CSVファイルのパス、エンコーディング、および列名を確認してください。")
    exit()

# --------------------------------------------------------

# 結果を保持するための最終的な階層型辞書
final_analysis_data = {}

# 投打の組み合わせによるループ
for pitcher_hand in df[PITCHER_HANDEDNESS].unique():
    df_pitcher = df[df[PITCHER_HANDEDNESS] == pitcher_hand]
    
    for batter_hand in df_pitcher[BATTER_HANDEDNESS].unique():
        df_combined = df_pitcher[df_pitcher[BATTER_HANDEDNESS] == batter_hand]
        
        data_key = f"{pitcher_hand}_{batter_hand}"
        final_analysis_data[data_key] = {}
        
        pitch_types_list = df_combined[df_combined[PITCH_TYPE_NAME].notna()][PITCH_TYPE_NAME].unique()
        
        for pitch_type in pitch_types_list:
            df_pitch = df_combined[df_combined[PITCH_TYPE_NAME] == pitch_type]
            
            # --- 1. 被打率 (battingAverage) の計算 ---
            batting_average_matrix = [[0.0] * 5 for _ in range(5)]
            
            # 🎯 修正点2: velocityData_list を25要素で初期化し、web_indexで格納する
            velocity_data_list = [None] * 25 
            
            # デバッグ出力（総投球数）
            print(f"\n=======================================================")
            print(f"組み合わせ: {data_key} | 球種: {pitch_type} | 総投球数: {len(df_pitch)}")
            print(f"=======================================================")
            
            # コースごとのループ
            for csv_zone_str, web_index in CSV_TO_WEB_INDEX_MAP.items():

                # CSVゾーン番号でフィルタリング (str.strip()済み)
                df_zone = df_pitch[df_pitch[PITCH_ZONE] == csv_zone_str]
                
                # 投球数 (打数) と安打数の計算
                total_at_bats = df_zone[IS_PITCH].sum() 
                total_hits = df_zone[IS_H].sum()
                total_swings = df_zone[IS_SWINGING_CUSTOM].sum()
                total_looks = df_zone[IS_LOOKING_CUSTOM].sum()
                
                # デバッグ出力（ゾーン別詳細）
                print(f"ZONE:{csv_zone_str} (Web Index:{web_index:2}) | データ数: {len(df_zone):5} | 打数合計: {total_at_bats:5} | 安打合計: {total_hits:5}")
                
                # ゾーンのデータが存在する場合のみ計算
                if total_at_bats > 0:
                    # Webアプリの行列位置を計算
                    row = web_index // 5
                    col = web_index % 5
                    
                    # 被打率の計算
                    batting_average = total_hits / total_at_bats
                    batting_average_matrix[row][col] = round(batting_average, 3)
                    
                    # --- 2. 球速別データ (velocityData) の計算 ---
                    speed_ranges = [f"{s}〜{s+9}" for s in range(60, 170, 10)] 
                    hits_by_speed = []
                    
                    for speed_start in range(60, 170, 10):
                        df_speed = df_zone[
                            (df_zone[PITCH_SPEED] >= speed_start) &
                            (df_zone[PITCH_SPEED] < speed_start + 10)
                        ]
                        hits_by_speed.append(len(df_speed)) 
                    
                    # --- 3. 見逃し・空振り比率の計算 ---
                    total_swings_looks = total_swings + total_looks 
                    
                    if total_swings_looks > 0:
                        looking_percent = round((total_looks / total_swings_looks) * 100)
                        swinging_percent = 100 - looking_percent
                    else:
                        looking_percent = 0
                        swinging_percent = 0
                        
                    # velocityDataリストに結果をWebインデックスの位置に格納
                    velocity_data_list[web_index] = {
                        "speeds": speed_ranges,
                        "hits": hits_by_speed, # 打数分布（投球数）
                        "looking": int(looking_percent),
                        "swinging": int(swinging_percent)
                    }
                else:
                    # データがゼロの場合もJSON構造を維持するために空のデータを格納
                    velocity_data_list[web_index] = {
                        "speeds": [f"{s}〜{s+9}" for s in range(60, 170, 10)],
                        "hits": [0] * 11,
                        "looking": 0,
                        "swinging": 0
                    }

            # 最終的なJSON構造に格納
            final_analysis_data[data_key][pitch_type] = {
                "battingAverage": batting_average_matrix,
                "velocityData": velocity_data_list
            }

# JSONファイルに出力
output_filename = "analysis_data.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(final_analysis_data, f, indent=4, ensure_ascii=False)

print(f"\n✅ ウェブアプリに必要な階層構造のデータが '{output_filename}' にJSON形式で保存されました。")
print("🚀 **コード改善点:** PITCH_ZONE の空白除去と、velocityData の Web Index 順での正確な格納処理を追加しました。")

/var/folders/6t/2qvj7grs6tx1rjydtydst6br0000gn/T/ipykernel_53652/2625460811.py:42: DtypeWarning: Columns (125,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/takurateruyoshi/Downloads/work_dataset/play_info.csv', encoding='cp932')


PITCH_ZONE unique values after stripping: ['2.0' '5.0' '8.0' '25.0' '3.0' '1.0' '16.0' '9.0' '23.0' 'nan' '4.0'
 '10.0' '18.0' '11.0' '12.0' '22.0' '6.0' '13.0' '20.0' '21.0' '7.0'
 '17.0' '14.0' '19.0' '24.0' '15.0']

組み合わせ: 右_左 | 球種: ストレート | 総投球数: 36388
ZONE:21.0 (Web Index: 0) | データ数:   561 | 打数合計:   561 | 安打合計:     3
ZONE:19.0 (Web Index: 1) | データ数:  1304 | 打数合計:  1304 | 安打合計:    30
ZONE:17.0 (Web Index: 2) | データ数:  1740 | 打数合計:  1740 | 安打合計:    60
ZONE:15.0 (Web Index: 3) | データ数:  2039 | 打数合計:  2039 | 安打合計:    51
ZONE:10.0 (Web Index: 4) | データ数:  2228 | 打数合計:  2228 | 安打合計:    16
ZONE:22.0 (Web Index: 5) | データ数:   703 | 打数合計:   703 | 安打合計:    13
ZONE:7.0 (Web Index: 6) | データ数:  1850 | 打数合計:  1850 | 安打合計:   140
ZONE:4.0 (Web Index: 7) | データ数:  2155 | 打数合計:  2155 | 安打合計:   197
ZONE:1.0 (Web Index: 8) | データ数:  2821 | 打数合計:  2821 | 安打合計:   203
ZONE:11.0 (Web Index: 9) | データ数:  1692 | 打数合計:  1692 | 安打合計:    30
ZONE:23.0 (Web Index:10) | データ数:   768 | 打数合計:   768 | 安打合計:    13
ZONE:8.0 (